## 第八章 特征提取

本章的特征提取指的是特征变换。

特征变换和前面的特征选择的目的都是将特征降维，但是特征选择是直接删掉一些特征，而特征提取是通过数学方法把高维特征映射为低维。

### 一、基于类别可分性判据的特征提取

这一节介绍的方法是通过最大化类内类间可分性判据$J$来对特征进行线性变换降维，也就是找到一个$\mathbf{W}$，将样本$\mathbf{x}$投影为$\mathbf{y = \mathbf{W}^T\mathbf{x}}$，用投影后的$\mathbf{y}$算出来的可分性判据$J$最大。

书上的推导过程需要先学会标量对矩阵求导，我在[补充的数学知识.ipynb](./补充的数学知识.ipynb)中有介绍。

经过一系列推导，最终的方法是：

1. 求出矩阵$\mathbf{S}_{\text{w}}^{-1}\mathbf{S}_{\text{b}}$。

关于$\mathbf{S}_{\text{w}}$和$\mathbf{S}_{\text{b}}$，书上在第四章和第七章中的表述不一致，我认为应该这里的公式应该采取第七章的说法，即

$$\begin{align}
\mathbf{S}_{\text{b}} &= \sum_{i=1}^cP_i(\mathbf{m}_i - \mathbf{m})(\mathbf{m}_i - \mathbf{m})^T \\
\mathbf{S}_{\text{w}} &= \sum_{i=1}^cP_i\frac{1}{n_i}\sum_{k=1}^{n_i}(\mathbf{x}_k^{(i)}-\mathbf{m}_i)(\mathbf{x}_k^{(i)}-\mathbf{m}_i)^T
\end{align}$$

其中$P_i$采用样本中类别的比例来进行估算，其实如果这样算的话公式里的$P_i\frac{1}{n_i}$就变成了$\frac{1}{n}$。

2. 求出$\mathbf{S}_{\text{w}}^{-1}\mathbf{S}_{\text{b}}$的特征值和特征向量。
3. 把特征值从大到小排序，选取前面最大的$d$个特征值的特征向量作为$\mathbf{W}$。$d$是人为划定的，你想把样本降维成几维，$d$就是多大。

计算特征值可以用`np.linalg.eig`方法来实现。`np.linalg.eig`返回值包含两部分，特征值和每个特征值对应的列向量（注意这里是列向量）。

接下来是代码实现，还是注意和前几章同样的问题，numpy中向量是行向量，代码中实现的矩阵是公式中矩阵的转置，除了`np.linalg.eig`返回的特征向量矩阵。

In [80]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

In [77]:
def get_mean(_X: np.ndarray) -> np.ndarray:
    """
    计算m_i（样本均值）
    :param _X: 样本矩阵，每一行是一个样本
    :return: 均值（一维向量）
    """
    return np.mean(_X, 0)


def get_within_class_scatter_matrix(_X: np.ndarray) -> np.ndarray:
    """
    计算某一类的S_w_i（类内离散度矩阵）
    :param _X: 样本矩阵，每一行是一个样本
    :return: 类内离散度矩阵（D * D）
    """
    ret = np.zeros((_X.shape[1], _X.shape[1]))
    m = get_mean(_X)
    for row in _X:
        ret += (row - m).reshape(m.shape[0], 1) @ (row - m).reshape(m.shape[0], 1).T
    return ret


def get_pooled_within_class_scatter_matrix(_X1: np.ndarray, _X2: np.ndarray) -> np.ndarray:
    """
    计算最终的S_w（总类内离散度矩阵）
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :return: 总类内离散度矩阵（D * D）
    """
    return get_within_class_scatter_matrix(_X1) + get_within_class_scatter_matrix(_X2) / (_X1.shape[0] + _X2.shape[0])

def get_between_class_scatter_matrix(_X1: np.ndarray, _X2: np.ndarray) -> np.ndarray:
    """
    计算S_b（类间离散度矩阵）
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :return: 类间离散度矩阵（D * D）
    """
    n1 = _X1.shape[0]
    n2 = _X2.shape[0]
    d = _X1.shape[1]
    N = n1 + n2
    m_1 = get_mean(_X1)
    m_2 = get_mean(_X2)
    m = get_mean(np.concatenate((_X1, _X2)))
    return n1 / N * ((m_1 - m).reshape(d, 1) @ (m_1 - m).reshape(d, 1).T) + \
        n2 / N * ((m_2 - m).reshape(d, 1) @ (m_2 - m).reshape(d, 1).T)

def get_W(_X1: np.ndarray, _X2: np.ndarray, d: int) -> np.ndarray:
    """
    计算最终的W矩阵
    :param _X1: 第一类样本矩阵，每一行是一个样本
    :param _X2: 第二类样本矩阵，每一行是一个样本
    :param d: 最终要投影的特征维度数量
    :return: W矩阵
    """
    S_w_inv = np.linalg.inv(get_pooled_within_class_scatter_matrix(_X1, _X2))
    S_b = get_between_class_scatter_matrix(_X1, _X2)
    c = np.linalg.eig(S_w_inv @ S_b)  # np.np.linalg.eig用来计算特征值和特征向量，返回的结果是复数

    # 下面用来对特征值排序，并且记录特征值的序号
    eigen_temp = []
    for i in range(c[0].shape[0]):
        # 找到是实数的特征值
        if c[0][i].imag == 0:
            eigen_temp.append([c[0][i].real, i])

    eigen_temp.sort(reverse=True)

    # 下面用来找出最大的d个实特征值的特征向量
    ret = []
    for i in range(d):
        ret.append(c[1][:,eigen_temp[i][1]].real)
    return np.array(ret)

In [81]:
# 加载数据，这次的数据用sklearn自带的水仙花的数据
iris = load_iris()
X = iris.data[:100]
y = iris.target[:100]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 寻找两类样本
X1 = []
X2 = []
for i in range(X_train.shape[0]):
    if y_train[i] == 0:
        X1.append(X_train[i])
    else:
        X2.append(X_train[i])
X1 = np.array(X1)
X2 = np.array(X2)

# 计算W, d设置为1，即投影后的新特征为1维
W = get_W(X1, X2, 1)
print("W为：")
print(W)

W为：
[[ 0.16495053 -0.33727579  0.8207165   0.43065159]]


接下来生成降维后的数据，用支持向量机训练测试一下效果。

In [83]:
X_train_new = X_train @ W.T
X_test_new = X_test @ W.T
print("降维之后的样本维度：", X_train_new.shape[1])

svm_classifier = SVC(kernel="rbf")  # 用径向基函数作为核函数
svm_classifier.fit(X_train_new, y_train)

print("降维之后的正确率：", svm_classifier.score(X_test_new, y_test))

降维之后的样本维度： 1
降维之后的正确率： 1.0


测试结果表明，用基于分类可分性判据的特征提取方法，就算我们把样本从四维降成一维，还能保持100%的正确率。这说明这个降维方法是非常有效的。

### 二、主成分分析方法

主成分分析是根据方差大小来对样本进行线性变换，最终实现对样本的降维。每一个主成分都是样本协方差矩阵的一个特征值。

主成分分析的公式推导并不复杂，书上介绍的也很清楚。

主要的问题在于主成分分析没有考虑样本分类信息，只是按照样本方差来处理数据，这样的方法只是把样本降维了，但是不一定对分类有利。

主成分分析实现过程和上面说的第一个方法有很多地方重叠，比如都要求特征值和特征向量，根据特征值大小排序，最后通过数学公式进行投影。因此我就不写代码实现了。

### 三、K-L变换